# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf #for deeplearning
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for image processing

In [2]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
img_height=64
img_width=64
batch_size=32

In [4]:
train_datagen = ImageDataGenerator( #train_dataget is an instance
        rescale=1./255, #feature scaling to each and every pixels
        #transformation
        shear_range=0.2, #we shear
        zoom_range=0.2, #we zoom
        horizontal_flip=True) #we flip

training_set = train_datagen.flow_from_directory( #using train_datagen we call the method flow_from_directory
        'dataset/training_set',#path to our dataset
         target_size=(img_height, img_width),
         batch_size=batch_size, #how many images in each batch
         class_mode='binary') #we are expecting binary outcome, cat and dog

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255) #feature scaling here.
#No transformation here

test_set = train_datagen.flow_from_directory(
        'dataset/test_set', #test set directory
        #all other remains same
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential() #cnn object created which be of the class Sequential

### Step 1 - Convolution

In [7]:
#we will add layers
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

C:\Users\shahr\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding a second convolutional layer

In [9]:
#copy of 1st convolutional layer with max pooling with removed input_shape as we already took the input earlier
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x=training_set, validation_data=test_set, epochs= 25 ) #applying the training set, we provide the test set in the valiadation data


Epoch 1/25


C:\Users\shahr\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 86ms/step - accuracy: 0.5426 - loss: 0.6886 - val_accuracy: 0.6400 - val_loss: 0.6234
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - accuracy: 0.6520 - loss: 0.6261 - val_accuracy: 0.6985 - val_loss: 0.5835
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - accuracy: 0.7149 - loss: 0.5709 - val_accuracy: 0.7060 - val_loss: 0.5685
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.7256 - loss: 0.5347 - val_accuracy: 0.7350 - val_loss: 0.5416
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.7481 - loss: 0.5084 - val_accuracy: 0.7575 - val_loss: 0.5216
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.7738 - loss: 0.4773 - val_accuracy: 0.7500 - val_loss: 0.5164
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.7955 - loss: 0.4453 - val_accuracy: 0.7815 - val_loss: 0.4764
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.7934 - loss: 0.4338 - val_accurac

## Part 4 - Making a single prediction

In [16]:
import numpy as np
from keras.preprocessing import image #importing image module
#load image from the folder
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) #dataset/single_prediction/cat_or_dog_1.jpg is the image location, target_size = (64, 64) [Note: Image size has to be same as we trained]
#predict method expects input in array . So, convert PIL image to Numpy array.
test_image = image.img_to_array(test_image)
#We have trained on batch. So, batch number 1 had 32 images, batch number 2 had 32 images and so on.... So, we need to add extra dimension batch
test_image = np.expand_dims(test_image, axis = 0)#adding a dimension to the test_image and on the 1st dimension.

result = cnn.predict(test_image/255.0) # we need to normalize

#if we get 1, it's a dog and if not, we will have a cat
training_set.class_indices #result has now the batch dimension and so, we will acess the first one using result[0] and then we will get access to only prediction within that by result[0][0]
if result[0][0] > 0.5:
#we are looking for the prediction on the basis of probability. So, if we cross more than 50%, it will be dog
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


In [17]:
print(prediction)

dog
